In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from tfinterface.model_base import ModelBase
from tfinterface.reinforcement import ExperienceReplay
from tfinterface.utils import select_columns, soft_if, get_run, map_gradients
from phi.api import *
import tensorflow as tf
import random
from scipy.interpolate import interp1d
import numpy as np
import gym
from gym import wrappers
from tfinterface.reinforcement import ExpandedStateEnv
import os
import time


name = "actor-critic-base"

In [3]:
class Inputs(object):
    def __init__(self, n_states, scope):
        with tf.variable_scope(scope):
            self.episode_length = tf.placeholder(tf.int64, [], name='episode_length')

            self.s = tf.placeholder(tf.float32, [None, n_states], name='s')
            self.a = tf.placeholder(tf.int32, [None], name='a')
            self.r = tf.placeholder(tf.float32, [None], name='r')
            self.v1 = tf.placeholder(tf.float32, [None], name='V1')
            self.done = tf.placeholder(tf.float32, [None], name='done')
            
            self.learning_rate = tf.placeholder(tf.float32, [], name='learning_rate')
            self.keep_prob = tf.placeholder(tf.float32, [], name='keep_prob')
            self.training = tf.placeholder(tf.bool, [], name='training')
            
            self.pi = tf.placeholder(tf.float32, [], name='pi')
            

class Critic(object):
    def __init__(self, base_model, inputs, n_actions, n_states, y, scope):
        with tf.variable_scope(scope):
            
            ops = dict(
                trainable=True,
                kernel_initializer=tf.random_uniform_initializer(minval=0.0, maxval=0.01),
                bias_initializer=tf.random_uniform_initializer(minval=0.0, maxval=0.01),
                use_bias=True
            )

            net = inputs.s

            net = tf.layers.dense(net, 128, activation=tf.nn.relu, name="relu_layer", **ops)
#             net = tf.nn.dropout(net, inputs.keep_prob)
            
            net = tf.layers.dense(net, 128, activation=tf.nn.relu, name="relu_layer2", **ops)
#             net = tf.nn.dropout(net, inputs.keep_prob)
            
            self.V = tf.layers.dense(net, 1, name='V', **ops)[:, 0]

            self.target = soft_if(inputs.done, inputs.r,  inputs.r + y * inputs.v1)

            self.error = self.target - self.V
            self.loss = Pipe(self.error, tf.nn.l2_loss, tf.reduce_mean)

            self.variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=scope)

#             self.update = tf.train.AdamOptimizer(inputs.learning_rate).minimize(self.loss, var_list=self.variables)
            trainer = tf.train.AdamOptimizer(inputs.learning_rate)
            gradients = trainer.compute_gradients(self.loss, var_list=self.variables)
            gradients = map_gradients(lambda g: tf.clip_by_norm(g, 1000), gradients)
            self.update = trainer.apply_gradients(gradients)
            
            avg_error, std_error = tf.nn.moments(self.error, [0])
            self.summaries = tf.summary.merge([
                tf.summary.scalar('loss', self.loss),
                tf.summary.scalar('avg_target', tf.reduce_mean(self.target)),
                tf.summary.scalar('variables_sum', sum([ tf.reduce_sum(v) for v in self.variables ])),
                tf.summary.scalar('avg_error', avg_error),
                tf.summary.scalar('std_error', std_error),
                tf.summary.histogram(
                    'avg_action', Pipe(
                    inputs.a,
                    Then(tf.one_hot, n_actions),
                    Then(tf.reduce_mean, axis=0)
                ))
            ]+[
                tf.summary.histogram('var{}'.format(i), self.variables[i]) for i in range(len(self.variables))
            ])
            
class Actor(object):
    def __init__(self, base_model, inputs, target_critic, n_actions, n_states, y, scope):
        with tf.variable_scope(scope):
            ops = dict(
                trainable=True,
                kernel_initializer=tf.random_uniform_initializer(minval=0.0, maxval=0.01),
                bias_initializer=tf.random_uniform_initializer(minval=0.0, maxval=0.01),
                use_bias=False
            )

            net = inputs.s

            net = tf.layers.dense(net, 128, activation=tf.nn.relu, name="relu_layer", **ops)
            net = tf.nn.dropout(net, inputs.keep_prob)
            
            net = tf.layers.dense(net, 128, activation=tf.nn.relu, name="relu_layer2", **ops)
            net = tf.nn.dropout(net, inputs.keep_prob)

            self.P = tf.layers.dense(net, n_actions, activation=tf.nn.softmax, name='P', **ops)
            self.Pa = select_columns(self.P, inputs.a)
            
            r_pred = net
#             r_pred = tf.concat([r_pred, tf.one_hot(inputs.a, n_actions)], 1)
#             r_pred = tf.layers.dense(net, 32, activation=tf.nn.softmax, name='r_pred_relu', **ops)
#             r_pred = tf.layers.dense(r_pred, 1, activation=tf.nn.softmax, name='r_pred', **ops)[:,0]
#             r_loss = 0.01 * Pipe(r_pred - inputs.r, tf.nn.l2_loss, tf.reduce_mean)

            self.loss = - tf.log(tf.clip_by_value(self.Pa, 1e-3, 1.0)) * target_critic.error
            self.loss = tf.reduce_mean(self.loss)

            self.variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=scope)

#             self.update = tf.train.AdamOptimizer(inputs.learning_rate).minimize(self.loss, var_list=self.variables)
            trainer = tf.train.AdamOptimizer(inputs.learning_rate)
            gradients = trainer.compute_gradients(self.loss, var_list=self.variables)
            gradients = map_gradients(lambda g: tf.clip_by_norm(g, 1000), gradients)
            self.update = trainer.apply_gradients(gradients)
            
            
            self.summaries = tf.summary.merge([
                tf.summary.scalar('loss', self.loss),
                tf.summary.scalar('variables_sum', sum([ tf.reduce_sum(v) for v in self.variables ])),
                tf.summary.histogram(
                    'avg_action', Pipe(
                    inputs.a,
                    Then(tf.one_hot, n_actions),
                    Then(tf.reduce_mean, axis=0)
                ))
            ]+[
                tf.summary.histogram('var{}'.format(i), self.variables[i]) for i in range(len(self.variables))
            ])
            


In [4]:
class LunarLander(ModelBase):
    
    def define_model(self, n_actions, n_states, y=0.98, buffer_length=50000, pi=0.1):
        self.global_max = float('-inf')

        self.positive_buffer = ExperienceReplay(max_length=buffer_length/2)
        self.negative_buffer = ExperienceReplay(max_length=buffer_length/2)


        with self.graph.as_default(), tf.device("cpu:0"):

            self.inputs = Inputs(n_states, "inputs")

            self.critic = Critic(self, self.inputs, n_actions, n_states, y, "critic")
            self.target_critic = Critic(self, self.inputs, n_actions, n_states, y, "target_critic")
            self.actor = Actor(self, self.inputs, self.target_critic, n_actions, n_states, y, "actor")

            self.update = tf.group(self.critic.update, self.actor.update)

            self.episode_length_summary = tf.summary.scalar('episode_length', self.inputs.episode_length)

            self.summaries = tf.summary.merge([self.actor.summaries, self.critic.summaries, self.target_critic.summaries])

            self.update_target = tf.group(*[
                t.assign_add(pi * (a - t)) for t, a in zip(self.target_critic.variables, self.critic.variables)
            ])
    
    
    def predict_feed(self, S):
        return {
            self.inputs.s: S,
            self.inputs.keep_prob: 1.0,
            self.inputs.training: False
        }
    
    def predict(self, state, e = 0.0):
        predict_feed = self.predict_feed([state])
        actions = self.sess.run(self.actor.P, feed_dict=predict_feed)
        actions = actions[0]
        n = len(actions)

        if random.random() < e:
            return random.randint(0, n-1)
        else:
            return np.random.choice(n, p=actions)
    
    def fit_feed(self, S, A, R, V1, Done, learning_rate, keep_prob):
        return {
            self.inputs.s: S,
            self.inputs.a: A,
            self.inputs.r: R,
            self.inputs.v1: V1,
            self.inputs.done: Done,
            self.inputs.learning_rate: learning_rate,
            self.inputs.keep_prob: keep_prob,
            self.inputs.training: True
        }
    
    
    def fit(self, env, keep_prob=0.5, e=0.01, learning_rate=0.01, print_step=10, 
            update_target_step = 32, episodes=100000, max_episode_length=float('inf'), batch_size=32,
            min_buffer_size=10000):
        
        r_total = 0.
        batch_size /= 2

        for episode in range(episodes):
            done = False
            ep_step = 0
            s = env.reset()
            episode_length = 0
            ep_reward = 0.
            
            while not done and ep_step <= max_episode_length:
                self.global_step += 1
                episode_length += 1
                ep_step += 1
                
                
                _learning_rate = learning_rate(self.global_step) if hasattr(learning_rate, '__call__') else learning_rate
                _e = e(self.global_step) if hasattr(e, '__call__') else e
                
                
                a = self.predict(s, e = _e)
                s1, r, done, info = env.step(a)
                r = min(max(-10., r), 10.)
                r_total += r
                ep_reward += r
                
                
                if r > 0:
                    self.positive_buffer.append((s, a, r, s1, float(done)))
                else:
                    self.negative_buffer.append((s, a, r, s1, float(done)))
                    
                s = s1
                
                
                
                Sp, Ap, Rp, S1p, Donep = self.positive_buffer.random_batch(batch_size).unzip() if len(self.positive_buffer) > 0 else ([], [], [], [], [])
                Sn, An, Rn, S1n, Donen = self.negative_buffer.random_batch(batch_size).unzip() if len(self.negative_buffer) > 0 else ([], [], [], [], [])
                
                S, A, R, S1, Done = Sp + Sn, Ap + An, Rp + Rn, S1p + S1n, Donep + Donen
                
                predict_feed = self.predict_feed(S1)
                V1 = self.sess.run(self.target_critic.V, feed_dict=predict_feed)

                
                fit_feed = self.fit_feed(S, A, R, V1, Done, _learning_rate, keep_prob)
                
                if self.global_step < min_buffer_size:
                    continue
                
                _, summaries = self.sess.run([self.update, self.summaries], feed_dict=fit_feed)
                self.writer.add_summary(summaries, self.global_step)
                
                
                if self.global_step % update_target_step == 0:
                    self.sess.run(self.update_target)
                
                
                
                
            
            episode_length_summary = self.sess.run(self.episode_length_summary,
                                                   feed_dict={self.inputs.episode_length: episode_length})
            self.writer.add_summary(episode_length_summary, self.global_step)


            if ep_reward >= self.global_max:
                print("[MAX] Episode: {}, Length: {}, Reward: {}, global_step: {}".format(episode, episode_length, ep_reward, self.global_step))
                self.save(model_path = self.model_path + ".{score}".format(score = ep_reward))
                self.global_max = ep_reward


            if episode % print_step == 0 and episode > 0:
                avg_r = r_total / print_step
                actor_loss = self.sess.run(self.actor.loss, feed_dict=fit_feed)
                print("[NOR] Episode: {}, Length: {}, Avg Reward: {}, e: {}, Learning Rate: {}, global_step: {}".format(episode, episode_length, avg_r, _e, _learning_rate, self.global_step))
                print("Loss: {}".format(actor_loss))
                self.save()
                r_total = 0.


In [16]:

env = gym.make("LunarLander-v2")
env = ExpandedStateEnv(env, 3)
n_actions = env.action_space.n
n_states = env.observation_space.shape[0] * 3
model_path =  "{path}/models/{run}".format(path = os.getcwd(), run = 75)
logs_path = "{path}/logs/".format(path = os.getcwd(), name = name)


model_run = LunarLander(
    n_actions, n_states,
    model_path = model_path,
    flush_secs = 3.0,
    restore = True
)

for i in range(100):
    s = env.reset()
    done = False
    total = 0.
    ep = 0
    while not done and ep < 700:
        ep += 1
        a = model_run.predict(s, 0.)
        s, r, done, info = env.step(a)
        total += r
        env.render()
        time.sleep(0.01)
    
    print(total)
    
env.render(close=True)


[2017-03-20 20:19:09,274] Making new env: LunarLander-v2


-70.9735026994
-74.0987000802
-54.3388840129
-8.19298049388
-29.1446292429
-35.2294793741
-54.9421207642
-88.8666497516
-85.8520058051
-47.1022662128
-14.0613811689
-22.3488412596
-99.5969495626
7.95981789916
-88.8510790949
-71.2578354585
-75.9225128488
-97.2410543979
-68.108061797
-79.3475218677
-55.5064504223
-10.9989818182
-92.6378808474
-41.9122755541
-91.4358301104
-28.1542538685
-71.0653980829
-75.0239781528
-63.997706834
-7.81119253799
-59.1140177305
-87.3226045044
-75.9597416635
-53.2034266146
-54.1755067129
-11.9173622045
-66.5861605856
-45.9250337228
-15.5142492584
-25.913664303
-44.7269481426
-72.0487683056
-55.2767374953
-61.0235000794
-23.3901501755
-58.6979741191
-70.5756089482
-87.4173823443
-56.5940433796
-59.0612326571
-89.314294611
-46.8570076758
-42.9373541291
-8.24847127384
-40.7610453435
-25.2036793355
-94.6075358144
-69.8322152382
-27.7779189039
-65.2888103261
-36.082711457
-62.0687482812
-82.9730800588
-68.0168367261
-76.5100144591
-61.1240075065
-1.36376967739
-

In [18]:
run

3